In [1]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-cg-toolkit' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-390' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-410' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-415' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common-418' for glob 'nvidia*'
Note, selecting 'nvidi

In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [25]:
%%time
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <chrono> 
#include <cassert>
using namespace std;
using namespace std::chrono; 

const int V = 1000;
  
const int INF = 99999;  

  
void InnerLoops (int dis[][V],int k)  
{  
	int tm = 0;
	for (int i = 0; i < V; i++)  
	{  
		for (int j = 0; j < V; j++)  
		{  
			tm = dis[i][k] + dis[k][j];
			dis[i][j] = tm*(tm < dis[i][j])+ dis[i][j]*(tm >= dis[i][j]);  
		}  
	}    
}  

void FloydWarshall(int dis[][V])
{
		for (int k = 0; k < V; k++)  
		{  
				InnerLoops(dis,k);
		}
}

int dis[V][V];
int main()  
{    
  for (int i = 0; i < V; i++) 
	{
		for(int j = 0; j < V; j++)
		{
			if(j==i+1) dis[i][j] = 1;
			else if (i!=j) dis[i][j] = INF;
			else dis[i][j] = 0;
		}
	}
	FloydWarshall(dis);
  for (int i = 0; i < V; i++) 
	{
		for(int j = 0; j < V; j++)
		{
			if(j>=i) assert(dis[i][j] == j-i);
			else assert(dis[i][j] == INF);
		}
	}
  return 0;  
} 


CPU times: user 2.42 ms, sys: 8.59 ms, total: 11 ms
Wall time: 9.92 s


In [24]:
%%time
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <chrono> 
#include <cassert>
using namespace std;
using namespace std::chrono; 

// Kernel function for inner two loop of Floyd Warshall Algorithm
__global__
void GPUInnerLoops(int V, int k, int *dis)
{
	//calculates unique thread ID in the block
	int t= (blockDim.x*blockDim.y)*threadIdx.z+(threadIdx.y*blockDim.x)+(threadIdx.x);
	//calculates unique block ID in the grid
	int b= (gridDim.x*gridDim.y)*blockIdx.z+(blockIdx.y*gridDim.x)+(blockIdx.x);
	//block size (this is redundant though)
	int T= blockDim.x*blockDim.y*blockDim.z;
	//grid size (this is redundant though)
	int B= gridDim.x*gridDim.y*gridDim.z;
	
  int tm = 0;
	/*
	 * Each cell in the matrix is assigned to a different thread. 
	 * Each thread do O(N*number of asssigned cell) computation.
	 * Assigned cells of different threads does not overlape with
	 * each other. And so no need for synchronization.
	 */
	 
    for (int i=b;i<V;i+=B)
    {
		  for(int j=t;j<V;j+=T)
		  {
			  tm = dis[i*V+k] + dis[k*V+j];
			  dis[i*V+j] = tm*(tm < dis[i*V+j])+ dis[i*V+j]*(tm >= dis[i*V+j]);
		  }
	  }
} 
  
const int V = 8000;
  
const int INF = 99999;  


void FloydWarshall(int *dis)
{
		for (int k = 0; k < V; k++)  
  	{
				GPUInnerLoops<<<dim3(16,8,8), dim3(8,8,8)>>>(V, k, dis);
				cudaDeviceSynchronize();
		}
}

int main(void)
{
	
	int *dis;

	// Allocate Unified Memory – accessible from CPU or GPU
	cudaMallocManaged(&dis, V*V*sizeof(int));

	// initialize dis array on the host
	for (int i = 0; i < V; i++) 
	{
		for(int j = 0; j < V; j++)
		{
			
			if(j==i+1) dis[i*V+j] = 1;
			else if (i!=j) dis[i*V+j] = INF;
			else dis[i*V+j] = 0;
		}
	}
  FloydWarshall(dis);
	
	for (int i = 0; i < V; i++) 
	{
		for(int j = 0; j < V; j++)
		{
			if(j>=i) 
	 		{
		 		assert( dis[i*V+j] == j-i);
			}
			else assert( dis[i*V+j] == INF);
		}
	}
  // Free memory
	cudaFree(dis);
	return 0;
}



CPU times: user 1.87 ms, sys: 8.23 ms, total: 10.1 ms
Wall time: 22.3 s
